# Gozumol - Colab Test Notebook

Bu notebook, Gozumol kütüphanesini Google Colab'da GPU ile test etmek için hazırlanmıştır.

**Adımlar:**
1. GPU'yu etkinleştir (Runtime > Change runtime type > GPU)
2. Hücreleri sırayla çalıştır

---

## 1. GPU Kontrolü

In [ ]:
import torch

if torch.cuda.is_available():
    print(f"GPU Aktif: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("GPU BULUNAMADI! Runtime > Change runtime type > GPU seçin!")

## 2. Gozumol Kurulumu

In [ ]:
# GitHub'dan repo'yu klonla
!git clone https://github.com/eminaruk/gozumol.git
%cd gozumol

# Bağımlılıkları yükle
!pip install -q torch transformers accelerate pillow requests

# Gozumol'u editable modda yükle
!pip install -q -e .

print("\nKurulum tamamlandı!")

## 3. Import Testi

In [ ]:
from gozumol import VisionAssistant
from gozumol.prompts import get_prompt_pair, QUICK_SCAN_USER_PROMPT
from gozumol.core.utils import get_device_info

# Device bilgisi
info = get_device_info()
print(f"Device: {info['device']}")
if info.get('cuda_device_name'):
    print(f"GPU: {info['cuda_device_name']}")
    print(f"VRAM: {info['cuda_memory_total_gb']:.1f} GB")

print("\nImport başarılı!")

## 4. Model Yükleme

**Not:** İlk çalıştırmada model indirilecek (~8GB). Bu birkaç dakika sürebilir.

In [ ]:
# VisionAssistant'ı oluştur (model yüklenecek)
print("Model yükleniyor... (ilk seferde ~5-10 dakika sürebilir)")

assistant = VisionAssistant(
    scenario='outdoor',  # Dış mekan senaryosu
    device='cuda'        # GPU kullan
)

print(f"\n{assistant}")
print("Model hazır!")

## 5. Test Görselleri Yükle

In [ ]:
import requests
from PIL import Image
from IPython.display import display

# Test görselleri
test_images = {
    "sokak_1": "https://heytripster.com/wp-content/uploads/2023/08/Trastevere-rome.jpg",
    "sokak_2": "https://oitheblog.com/wp-content/uploads/2016/03/SAM_7237.jpg",
    "kalabalik": "https://images.unsplash.com/photo-1517732306149-e8f829eb588a?w=800",
}

images = {}
for name, url in test_images.items():
    try:
        img = Image.open(requests.get(url, stream=True, timeout=10).raw)
        images[name] = img
        print(f"{name}: {img.size} - Yüklendi")
    except Exception as e:
        print(f"{name}: Yüklenemedi - {e}")

print(f"\n{len(images)} görsel yüklendi!")

In [ ]:
# Görselleri göster
for name, img in images.items():
    print(f"\n--- {name} ---")
    display(img.resize((400, 300)))

## 6. Test: Temel Çevre Açıklaması

Varsayılan `outdoor` senaryosu ile test.

In [ ]:
# İlk görsel ile test
print("Görsel analiz ediliyor...\n")

response, timing = assistant.describe(images["sokak_1"], return_timing=True)

print("=" * 60)
print("ÇEVRE AÇIKLAMASI")
print("=" * 60)
print(response)
print("=" * 60)
print(f"\nSüre: {timing['total_time']:.2f}s")
print(f"  - İşleme: {timing['processor_time']:.2f}s")
print(f"  - Üretim: {timing['generation_time']:.2f}s")
print(f"  - Decode: {timing['decode_time']:.2f}s")

## 7. Test: Hızlı Güvenlik Taraması

`quick_scan()` metodu - sadece kritik güvenlik bilgisi (2-3 cümle).

In [ ]:
print("Hızlı güvenlik taraması...\n")

quick_response, timing = assistant.quick_scan(images["sokak_2"], return_timing=True)

print("=" * 60)
print("HIZLI GÜVENLİK TARAMASI")
print("=" * 60)
print(quick_response)
print("=" * 60)
print(f"\nSüre: {timing['total_time']:.2f}s")

## 8. Test: Karşıdan Karşıya Geçiş

`check_crossing()` metodu - trafik durumu ve geçiş güvenliği.

In [ ]:
print("Karşıdan karşıya geçiş kontrolü...\n")

crossing_response, timing = assistant.check_crossing(images["sokak_1"], return_timing=True)

print("=" * 60)
print("GEÇİŞ GÜVENLİĞİ")
print("=" * 60)
print(crossing_response)
print("=" * 60)
print(f"\nSüre: {timing['total_time']:.2f}s")

## 9. Test: Farklı Senaryolar

Aynı görsel üzerinde farklı senaryoları karşılaştır.

In [ ]:
scenarios_to_test = ['outdoor', 'traffic', 'quick', 'detailed']
test_image = images["sokak_2"]

print("Farklı senaryolarla test ediliyor...\n")
display(test_image.resize((400, 300)))

results = {}
for scenario in scenarios_to_test:
    assistant.set_scenario(scenario)
    response, timing = assistant.describe(test_image, return_timing=True)
    results[scenario] = {'response': response, 'time': timing['total_time']}
    print(f"\n{'='*60}")
    print(f"SENARYO: {scenario.upper()} ({timing['total_time']:.2f}s)")
    print(f"{'='*60}")
    print(response)

## 10. Kendi Görselini Test Et

Kendi görselini yükle ve test et.

In [ ]:
from google.colab import files

print("Görsel yüklemek için aşağıdaki butona tıkla:")
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    custom_image = Image.open(filename)
    print(f"\nYüklenen: {filename} ({custom_image.size})")
    display(custom_image.resize((400, 300)))

In [ ]:
# Yüklenen görseli test et
if 'custom_image' in dir():
    assistant.set_scenario('outdoor')
    
    print("Görsel analiz ediliyor...\n")
    response, timing = assistant.describe(custom_image, return_timing=True)
    
    print("=" * 60)
    print("ÇEVRE AÇIKLAMASI")
    print("=" * 60)
    print(response)
    print("=" * 60)
    print(f"\nSüre: {timing['total_time']:.2f}s")
else:
    print("Önce yukarıdaki hücrede bir görsel yükle!")

## 11. URL'den Görsel Test Et

In [ ]:
# Buraya istediğin görsel URL'sini yapıştır
image_url = "https://images.unsplash.com/photo-1477959858617-67f85cf4f1df?w=800"

# Görseli indir ve göster
url_image = Image.open(requests.get(image_url, stream=True).raw)
display(url_image.resize((400, 300)))

# Analiz et
assistant.set_scenario('outdoor')
response, timing = assistant.describe(url_image, return_timing=True)

print("\n" + "=" * 60)
print("ÇEVRE AÇIKLAMASI")
print("=" * 60)
print(response)
print("=" * 60)
print(f"\nSüre: {timing['total_time']:.2f}s")

## 12. Performans Özeti

In [ ]:
# GPU bellek kullanımı
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated() / 1024**3
    reserved = torch.cuda.memory_reserved() / 1024**3
    total = torch.cuda.get_device_properties(0).total_memory / 1024**3
    
    print("GPU Bellek Kullanımı:")
    print(f"  - Ayrılmış: {allocated:.2f} GB")
    print(f"  - Rezerve: {reserved:.2f} GB")
    print(f"  - Toplam: {total:.2f} GB")
    print(f"  - Kullanım: {(reserved/total)*100:.1f}%")

print("\nTest Tamamlandı!")

## 13. Temizlik (Opsiyonel)

GPU belleğini temizle.

In [ ]:
# GPU belleğini temizle
assistant.clear_memory()

# Model'i bellekten kaldır (opsiyonel)
# del assistant
# torch.cuda.empty_cache()

print("Bellek temizlendi!")